In [4]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-

# terminal: $ python3 from_mxnet.py -p test_data_path -t target

import mxnet as mx
from mxnet.gluon.model_zoo import vision
from mxnet import nd
import tvm
import tvm.relay as relay
from tvm.contrib import graph_runtime 

import numpy as np
# import matplotlib.pyplot as plt
import time
import os
import glob
from PIL import Image 
import argparse
import sys

img_list = []


def transform_image_np(image):
    image = image.transpose((2, 0, 1))
    image = np.expand_dims(image, axis=0)
    rgb_mean = np.array([0.485, 0.456, 0.406]).reshape((1, 3, 1, 1))
    rgb_std = np.array([0.229, 0.224, 0.225]).reshape((1, 3, 1, 1))
    return (image.astype('float32') / 255 - rgb_mean) / rgb_std
    

def get_test_data(test_data_path, max=1):
    # test data
    count = 0

    img_paths = glob.glob(os.path.join(test_data_path, "*.JPEG"))
    img_paths.sort()

    for path in img_paths:
        count += 1
        if count > max:
            break
    
        # Images under image files, either single channel (Black and White) or triple channel (RGB).
        # Unify the format of read images to prevent “ValueError: axes don't match array ”
        np_image = Image.open(path).convert("RGB").resize((256, 256))
        np_img = np_image.crop((16, 16, 240, 240))
        img_list.append(np.array(np_img))
    #     plt.imshow(np.array(np_img))
    #     plt.show()

In [5]:
# download pre-trained model from mxnet model_zoo
block = vision.get_model('MobileNet1.0', pretrained=True)
    
# # ImageNet Label
# # Synset for converting the number of ImageNet classes to human vocabulary
# synset_path = "./imagenet1000_clsid_to_human.txt"

# with open(synset_path) as f:
#     text_labels = eval(f.read())
        
get_test_data('/Users/Mac_Chen/Desktop/TVM/from_tvm_to_deeplearning/tst')
    
    
print('Relay: get model from mxnet...')

img_ = transform_image_np(img_list[0])
print('img', img_.shape, 'type: ', type(img_))

shape_dict = {'data': img_.shape}
print('Block: {0}, Dict_shape: {1}'.format(type(block), type(shape_dict)))

Relay: get model from mxnet...
img (1, 3, 224, 224) type:  <class 'numpy.ndarray'>
Block: <class 'mxnet.gluon.model_zoo.vision.mobilenet.MobileNet'>, Dict_shape: <class 'dict'>


In [6]:
mod, params = relay.frontend.from_mxnet(block, shape_dict)
print('Mod: {0}, Params: {1}'.format(type(mod), type(params)))
func = mod['main']
func = relay.Function(func.params, relay.nn.softmax(func.body), None, func.type_params, func.attrs)
# print("Relay: build the graph")
# # target = 'llvm'
# if argv_target == 'llvm':
#     target = tvm.target.create('llvm')
#     ctx = tvm.cpu(0)
# elif argv_target == 'cuda':
#     target = tvm.target.create('cuda')
#     ctx = tvm.gpu(0)
# else:
#     target = argv_target


target = tvm.target.create('llvm')
ctx = tvm.cpu(0)

with relay.build_config(opt_level=3):
    graph, lib, params = relay.build(func, target, params=params)  
print("Graph: {0}, lib: {1}, params: {2}".format(type(graph), type(lib), type(params)))

Mod: <class 'tvm.ir.module.IRModule'>, Params: <class 'dict'>


Cannot find config for target=llvm, workload=('conv2d_NCHWc.x86', ('TENSOR', (1, 3, 224, 224), 'float32'), ('TENSOR', (32, 3, 3, 3), 'float32'), (2, 2), (1, 1, 1, 1), (1, 1), 'NCHW', 'NCHW', 'float32'). A fallback configuration is used, which may bring great performance regression.
Cannot find config for target=llvm, workload=('depthwise_conv2d_NCHWc.x86', ('TENSOR', (1, 32, 112, 112), 'float32'), ('TENSOR', (32, 1, 3, 3), 'float32'), (1, 1), (1, 1, 1, 1), (1, 1), 'NCHW', 'NCHW', 'float32'). A fallback configuration is used, which may bring great performance regression.
Cannot find config for target=llvm, workload=('conv2d_NCHWc.x86', ('TENSOR', (1, 32, 112, 112), 'float32'), ('TENSOR', (64, 32, 1, 1), 'float32'), (1, 1), (0, 0, 0, 0), (1, 1), 'NCHW', 'NCHW', 'float32'). A fallback configuration is used, which may bring great performance regression.
Cannot find config for target=llvm, workload=('depthwise_conv2d_NCHWc.x86', ('TENSOR', (1, 64, 112, 112), 'float32'), ('TENSOR', (64, 1, 3

Graph: <class 'str'>, lib: <class 'tvm.runtime.module.Module'>, params: <class 'dict'>


# Export graph, lib, params

In [24]:
tmp_path = '/Users/Mac_Chen/Desktop/'
local_lib_path = os.path.join(tmp_path, "deploy_lib.tar")
local_graph_path = os.path.join(tmp_path, "deploy_graph.json")
local_params_path = os.path.join(tmp_path, "deploy_params.params")


lib.export_library(local_lib_path)

with open(local_graph_path, 'w') as fo:
    fo.write(graph)

with open(local_params_path, 'wb') as fo:
    fo.write(relay.save_param_dict(params))

# Download(ftp) to JetsonNano

In [27]:
import pysftp
device_ip = "100.89.224.107"
device_uname = "terry"
device_pwd = "nvidia"
remote_path = "/home/terry/Desktop/sftp"

cnopts = pysftp.CnOpts()
cnopts.hostkeys = None   
with pysftp.Connection(device_ip, username=device_uname, password=device_pwd, cnopts=cnopts) as sftp:
    with sftp.cd(remote_path):
        sftp.put(local_lib_path)
        sftp.put(local_graph_path)
        sftp.put(local_params_path)

# Reload graph, lib, params

In [9]:
loaded_json = open(os.path.join(tmp_path, "deploy_graph.json")).read()
loaded_lib = tvm.runtime.load_module(os.path.join(tmp_path, "deploy_lib.tar"))
loaded_params = bytearray(open(os.path.join(tmp_path, "deploy_params.params"), 'rb').read())
print(type(loaded_json), '\n', type(loaded_lib), '\n', type(loaded_params))

<class 'str'> 
 <class 'tvm.runtime.module.Module'> 
 <class 'bytearray'>


In [11]:
print('Tvm: run the graph')
ctx = tvm.cpu(0)
dtype = 'float32'
m = graph_runtime.create(loaded_json, loaded_lib, ctx)
    
print('Input the img')
start_time_tvm = time.time()
prob_avg = 0
count = 0
    
for img_ in img_list:
    count += 1
    m.load_params(loaded_params)
    m.set_input('data', tvm.nd.array(transform_image_np(img_).astype(dtype)))
    m.run()
    
    tvm_output = m.get_output(0)
    tvm_output = tvm_output.asnumpy()[0]

    idx = np.argsort(tvm_output)[-3:][::-1]
    #     print('With prob = %.5f, it contains %s' % (tvm_output[idx[0]], text_labels[idx[0]]))
    
    prob_avg += tvm_output[idx[0]]

print('Average accuracy = %0.5f' % float(prob_avg / count))
print('Cost of time: %.5f sec' % (time.time() - start_time_tvm))


Tvm: run the graph
Input the img
Average accuracy = 0.37079
Cost of time: 0.12177 sec
